# 외부 데이터인 Tour API와 주어진 관광지 데이터 병합

In [1]:
import pandas as pd
import json

- 병하 파일 받아야함

In [2]:
mct = pd.read_csv('./data/MCT_processed.csv')
mct

,YM,MCT_NM,OP_YMD,MCT_TYPE,ADDR,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,...,평일 이용건수,주말 이용건수,현지인,남성,여성,20대이하,30대,40대,50대,60대이상
0,202301,통큰돼지,20110701,가정식,제주 제주시 용담이동 2682-9번지 통큰돼지,2_10~25%,3_25~50%,4_50~75%,0.161290,0.032258,...,52.258065,37.741935,63.698630,54.90,35.10,9.27,11.16,22.05,34.83,12.78
1,202301,해변,20050407,단품요리 전문,제주 제주시 애월읍 애월리 410-6번지,4_50~75%,5_75~90%,5_75~90%,0.090909,0.121212,...,40.303030,29.696970,16.164948,37.94,32.06,15.47,14.07,13.65,17.08,9.73
2,202301,한그릇,20120919,단품요리 전문,제주 서귀포시 색달동 2315-1번지 한그릇,4_50~75%,4_50~75%,3_25~50%,0.224719,0.112360,...,40.112360,29.887640,24.294118,37.03,32.97,9.66,21.63,23.94,11.48,3.29
3,202301,협재해녀의집,20130627,가정식,제주 제주시 한림읍 협재리 1459-2번지,4_50~75%,4_50~75%,3_25~50%,0.000000,0.271739,...,47.934783,22.065217,1.126687,33.88,36.12,22.61,22.47,14.63,7.63,2.73
4,202301,까망꼬숑,20220826,단품요리 전문,제주 제주시 노형동 3784-12번지 1층,2_10~25%,4_50~75%,5_75~90%,0.272727,0.121212,...,68.181818,21.818182,23.801653,46.17,43.83,37.08,27.99,15.84,6.84,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67921,202312,미나리,20230630,가정식,제주 제주시 연동 291-24번지 연동,4_50~75%,5_75~90%,3_25~50%,0.192000,0.216000,...,70.000000,0.000000,50.726351,38.22,31.78,6.37,18.27,21.91,16.87,6.58
67922,202312,종달리엔심야식당,20210513,가정식,제주 제주시 구좌읍 종달리 1935-1번지 1층,3_25~50%,3_25~50%,4_50~75%,0.150000,0.150000,...,38.000000,42.000000,1.104536,20.08,59.92,40.32,28.08,6.16,4.16,1.36
67923,202312,경일낙지,20020112,단품요리 전문,제주 제주시 노형동 1291-3번지,5_75~90%,5_75~90%,3_25~50%,0.179310,0.241379,...,60.000000,0.000000,49.504950,25.44,34.56,3.24,13.08,22.74,15.78,5.16
67924,202312,흑돼지가있는풍경,20040109,가정식,제주 제주시 노형동 668번지,2_10~25%,5_75~90%,6_90% 초과,0.066667,0.000000,...,42.000000,48.000000,17.701422,51.66,38.34,9.81,16.92,19.17,27.45,16.65


# 관광지 데이터 전처리

In [3]:
tour_sentiment = pd.read_csv('data/제주_관광_데이터/제주 관광지 평점리뷰 감성분석 데이터/JT_PORTAL_SITE_AVRG_SCORE_REVIEW_STANL_INFO_202306.csv')
tour_element = pd.read_csv('data/제주_관광_데이터/제주 관광지 평점리뷰 형태소 데이터/JT_PORTAL_SITE_AVRG_SCORE_REVIEW_MOP_INFO_202301.csv')
tour_month = pd.read_csv('data/제주_관광_데이터/제주 관광수요 예측 데이터_비짓제주 월별 데이터/JT_MT_ACCTO_TRRSRT_SCCNT_LIST_202301.csv')
tour_week = pd.read_csv('data/제주_관광_데이터/제주 관광수요 예측 데이터_비짓제주 요일별 데이터/JT_WKDAY_ACCTO_TRRSRT_SCCNT_LIST_202301.csv')

- 제공받은 관광지 데이터에서 관광지명 추출, 중복값 제거

In [7]:
tour_sentiment_name = tour_sentiment['TRRSRT_NM']
tour_element_name = tour_element['TRRSRT_NM']
tour_month_name = tour_month['AREA_NM']
tour_week_name = tour_week['AREA_NM']
whole_tour_name = pd.concat([tour_sentiment_name, tour_element_name, tour_month_name, tour_week_name])
whole_tour_name = whole_tour_name.drop_duplicates()

In [9]:
whole_tour_name.reset_index(drop=True, inplace=True)

In [10]:
# 외부 데이터 관광지명 추출

with open('openapi/jeju_total.json','r') as f:
    api_jeju = json.load(f)
api_name = []
for k,v in api_jeju['contents'].items():
    api_name.append(k)

# 관광지 데이터와 외부 관광지 데이터 관광지명 중복값 추출

In [12]:
import re

def find_text_between_special_characters(text):
    pattern = r'(\(|\[)([^\(\)\[\]]+?)(\)|\])'
    matches = re.findall(pattern, text)
    return [''.join(match) for match in matches]

In [13]:
whole_tour_name = [str(x) for x in whole_tour_name]

for i in range(len(whole_tour_name)):
    result = find_text_between_special_characters(whole_tour_name[i])
    if result:
        whole_tour_name[i] = whole_tour_name[i].replace(result[0],'')
    whole_tour_name[i] = whole_tour_name[i].replace(' ', '')
        

In [14]:
dup_name = []
for name in whole_tour_name:
    if name in api_name:
        dup_name.append(name)

In [15]:
# 총 863개 겹침
len(dup_name)

863

In [8]:
dup_tour_week = tour_week.loc[tour_week['AREA_NM'].apply(lambda x: x in dup_name)]

In [79]:
tour_week.CL_NM.value_counts()

CL_NM
음식점    1724
관광지    1143
숙박      975
쇼핑      140
Name: count, dtype: int64

In [70]:
dup_tour_week.CL_NM.value_counts()

CL_NM
음식점    303
관광지    299
숙박      64
쇼핑      19
Name: count, dtype: int64

- 병하 데이터 불러오기

In [38]:
tour_emo = pd.read_csv('data/tour_emo.csv')
tour_month = pd.read_csv('data/tour_month.csv')
tour_sylab = pd.read_csv('data/tour_sylab.csv')
tour_week = pd.read_csv('data/tour_week.csv')

tour_emo['TRRSRT_NM'] = tour_emo['TRRSRT_NM'].apply(lambda x: x.replace(find_text_between_special_characters(x)[0],'') if find_text_between_special_characters(x) else x)
tour_emo['TRRSRT_NM'] = tour_emo['TRRSRT_NM'].apply(lambda x: x.replace(' ', ''))
tour_sylab['TRRSRT_NM'] = tour_sylab['TRRSRT_NM'].apply(lambda x: x.replace(find_text_between_special_characters(x)[0],'') if find_text_between_special_characters(x) else x)
tour_sylab['TRRSRT_NM'] = tour_sylab['TRRSRT_NM'].apply(lambda x: x.replace(' ', ''))
tour_month['AREA_NM'] = tour_month['AREA_NM'].apply(lambda x: x.replace(find_text_between_special_characters(x)[0],'') if find_text_between_special_characters(x) else x)
tour_month['AREA_NM'] = tour_month['AREA_NM'].apply(lambda x: x.replace(' ', ''))
tour_week['AREA_NM'] = tour_week['AREA_NM'].apply(lambda x: x.replace(find_text_between_special_characters(x)[0],'') if find_text_between_special_characters(x) else x)
tour_week['AREA_NM'] = tour_week['AREA_NM'].apply(lambda x: x.replace(' ', ''))


In [39]:
#tour_emo = tour_emo.loc[tour_emo['TRRSRT_NM'].apply(lambda x: x in dup_name)]
#tour_sylab = tour_sylab.loc[tour_sylab['TRRSRT_NM'].apply(lambda x: x in dup_name)]

# 관광지 감성분석과 형태소 분석 데이터는 관광지명 종류가 매우 적으므로 제외함.

tour_month = tour_month.loc[tour_month['AREA_NM'].apply(lambda x: x in dup_name)]
tour_week = tour_week.loc[tour_week['AREA_NM'].apply(lambda x: x in dup_name)]


In [18]:
api_jeju_df = pd.DataFrame(api_jeju['contents']).T

In [19]:
api_jeju_df.reset_index(inplace=True)

In [20]:
api_jeju_df.rename(columns={'index':'AREA_NM'},inplace=True)

In [21]:
api_jeju_df['AREA_NM'] = api_jeju_df['AREA_NM'].apply(lambda x: x.replace(find_text_between_special_characters(x)[0],'') if find_text_between_special_characters(x) else x)
api_jeju_df['AREA_NM'] = api_jeju_df['AREA_NM'].apply(lambda x: x.replace(' ', ''))

In [40]:
api_jeju_df = api_jeju_df.loc[api_jeju_df['AREA_NM'].apply(lambda x: x in dup_name)]
#tour_emo = tour_emo.loc[tour_emo['TRRSRT_NM'].apply(lambda x: x in dup_name)]
#tour_sylab = tour_sylab.loc[tour_sylab['TRRSRT_NM'].apply(lambda x: x in dup_name)]
tour_month = tour_month.loc[tour_month['AREA_NM'].apply(lambda x: x in dup_name)]
tour_week = tour_week.loc[tour_week['AREA_NM'].apply(lambda x: x in dup_name)]

In [24]:
api_jeju_df.drop(columns=['booktour'],inplace=True)

In [53]:
tour_month = tour_month.groupby(['AREA_NM']).sum().reset_index()

In [54]:
tour_month.drop(columns=['CL_CD','CL_NM','ADDR','BASE_YEAR'],inplace=True)

In [63]:
tour_month.rename(columns={'ALL_TOTAL_CO': '총 방문자수'},inplace=True)

### 외부데이터와 관광지데이터 겹치는 데이터 병합, 
### 월별, 요일별 데이터 병합
### 컬럼명 변경

In [92]:
dfs = {}
for i in range(1,13):
    dfs[f'tour_week_{i}'] = tour_week.loc[tour_week['BASE_MT'] == i]
    dfs[f'tour_week_{i}'].rename(columns={'ALL_TOTAL_CO':f'{i}월 방문자수'},inplace=True)


/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_6182/3667516696.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[f'tour_week_{i}'].rename(columns={'ALL_TOTAL_CO':f'{i}월 방문자수'},inplace=True)


In [93]:
for i in range(1,13):
    dfs[f'tour_week_{i}'] = pd.merge(dfs[f'tour_week_{i}'],tour_month.iloc[:,[0,1]],on='AREA_NM',how='left')
    dfs[f'tour_week_{i}'].drop(columns=['CL_CD','CL_NM','ADDR','BASE_YEAR'],inplace=True)
    dfs[f'tour_week_{i}'] = pd.merge(api_jeju_df,dfs[f'tour_week_{i}'],on='AREA_NM',how='left')
    dfs[f'tour_week_{i}'].rename(columns={'AREA_NM':'관광지명','addr':'주소','contenttypeid':'장소 카테고리','cat1':'대분류',
                                          'cat2':'중분류','cat3':'소분류','BASE_MT':'기준월','MON_VIEWS_CO':'월요일방문자수','TUES_VIEWS_CO':'화요일방문자수','WED_VIEWS_CO':'수요일방문자수',
                                            'THUR_VIEWS_CO':'목요일방문자수','FRI_VIEWS_CO':'금요일방문자수','SAT_VIEWS_CO':'토요일방문자수','SUN_VIEWS_CO':'일요일방문자수'  },inplace=True)
    dfs[f'tour_week_{i}'] = dfs[f'tour_week_{i}'][['관광지명', '주소', '장소 카테고리', '대분류', '중분류', '소분류', '총 방문자수','기준월', f'{i}월 방문자수',
                                                  '월요일방문자수', '화요일방문자수', '수요일방문자수', '목요일방문자수', '금요일방문자수', '토요일방문자수',
                                                  '일요일방문자수']]
    dfs[f'tour_week_{i}'].dropna(inplace=True)
    dfs[f'tour_week_{i}'].to_csv(f'data/tour_df/original/tour_{i}.csv',index=False)